# Análise tabela de ocorrências

Este notebook tem como objetivo realizar uma análise detalhada das ocorrências ferroviárias registradas em um dataset específico. A análise inclui a importação e leitura dos dados, verificação de tipos de dados, análise descritiva e identificação de colunas com valores nulos.

## Estrutura do Notebook

1. **Importação de Bibliotecas Necessárias**: Importamos as bibliotecas essenciais para manipulação e visualização dos dados.
2. **Leitura do Arquivo**: Carregamos o arquivo CSV contendo os dados das ocorrências ferroviárias.
3. **Análise Colunar**: Realizamos uma análise rápida para descrever o conteúdo das colunas.
4. **Verificação de Tipos de Dados**: Verificamos os tipos de dados presentes em cada coluna.
5. **Análise Descritiva**: Calculamos estatísticas descritivas para entender melhor a distribuição dos dados.
6. **Identificação de Colunas Nulas**: Identificamos colunas que possuem apenas valores nulos, o que pode ser útil para futuras etapas de limpeza de dados.

## Objetivo

O objetivo principal é entender a estrutura dos dados, identificar possíveis problemas como colunas vazias e preparar o dataset para análises mais aprofundadas no futuro.

## Importação de bibliotecas necessárias

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


## Lendo arquivo

Nessa etapa, teremos que carregar o arquivo em uma váriavel, com isso verificando se há algum erro no csv.

In [ ]:
df = pd.read_csv(os.path.join("..", "dados", "TABELA_BIG_DATA_FT_OCORRENCIAS.csv"), encoding="utf-16le")

df.head()

In [ ]:
df[df["Classificacao Manchete"] == "USUÁRIO"]["Desc Ocorrencia"].head()

#salvando em um arquivo
df[df["Classificacao Manchete"] == "USUÁRIO"]["Desc Ocorrencia"].to_csv("ocorrencias_usuario.csv", index=False)

In [ ]:
tipo_relevancia = df["Tx Tipo Relevancia"].value_counts()

tipo_relevancia

## Análise da tabela

### Análise colunar

Trata-se de uma análise rápida que irá apenas descrever rapidamente o conteudo das colunas

Podemos primeiramente verificar os tipos dos valores nas colunas, quantidades 

In [ ]:
df.info()

Agora vemos a distribuição, media e outras caracteristicas das colunas

In [ ]:
df.describe()

Com isso, verificamos que algumas colunas possuem apenas valores nulos, o que pode ser comprovado com a seguinte célula

In [ ]:
def check_null_cols(df):
    n_rows = df.shape[0]
    df_nulls = df.isnull().sum()
    only_null_cols = df_nulls[df_nulls == n_rows].index
    if len(only_null_cols) > 0:
        print(f"Colunas com todos os valores nulos: {list(only_null_cols)}")
    else:
        print("Não há colunas com todos os valores nulos")

check_null_cols(df)


O código acima compara o numero de linhas no df com o numero de linhas nulas em cada coluna. Se o numero de linhas nulas na coluna for igual ao numero de linhas no df, então temos uma coluna vazia. Nesse caso, encontramos 3 colunas vazias: Data Atualizacao, Eventos Relacionados e Tx Trem, algo que pode ser levado em conta para passos futuros, como uma limpeza de dados.

Agora podemos verificar a distribuição dos valores em cada uma das colunas que forem relacionadas a alguma classificação, isso deverá ser deduzido a partir do nome da coluna, a fim de entender como os dados estão reunidos e se estão centralizados em algum valor

In [ ]:
df.columns

## Análise Coluna Categóricas

Com isso, temos os nomes das colunas e, observando a tabela e também o nome da coluna, conseguimos deduzir que Class Manchete e Classificacao Manchete são colunas categóricas, ou seja, não serão dados de uma data, numéricos ou etc, mas sim trarão alguma característica para os dados.

Com isso, a fim de deixar nosso código mais reaproveitavel, a seguinte função gera um grafico de barras sobre a frequência de dados em uma coluna da tabela

In [ ]:
def gen_value_counts_graph(df, col):
    value_counts = df[col].value_counts()
    
    plt.figure(figsize=(10, 6))
    plt.bar(value_counts.index, value_counts.values)
    plt.title(f"Contagem de valores para a coluna {col}")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()
    
    print(f"Contagem de valores para a coluna {col}:")
    print(value_counts)
    print("\n")

Com isso, conseguimos realizar a contagem para a coluna Class Manchete

In [ ]:
gen_value_counts_graph(df, "Class Manchete")

Agora para a coluna Classificacao Manchete

In [ ]:
gen_value_counts_graph(df, "Classificacao Manchete")

Apenas visualmente, vemos que a maior parte das ocorrências nas duas classificações são acerca de Usuários e Segurança Pública. Porém podemos provar isso com o Príncipio de pareto

### Principio de pareto

### Princípio de Pareto

O Princípio de Pareto, também conhecido como regra 80/20, é um conceito que sugere que, para muitos eventos, aproximadamente 80% dos efeitos vêm de 20% das causas. Esse princípio foi nomeado em homenagem ao economista italiano Vilfredo Pareto, que observou que 80% das terras na Itália eram possuídas por 20% da população.

No contexto da análise de dados, o Princípio de Pareto pode ser utilizado para identificar quais fatores ou categorias têm o maior impacto em um determinado resultado. Por exemplo, ao analisar ocorrências ferroviárias, podemos descobrir que 80% dos incidentes são causados por 20% dos fatores identificados. Isso pode ajudar a direcionar esforços e recursos para as áreas que terão o maior impacto na melhoria dos resultados.

A aplicação do Princípio de Pareto pode ser visualizada através de gráficos de Pareto, que são gráficos de barras que mostram a frequência ou impacto de diferentes categorias, juntamente com uma linha que representa a contribuição cumulativa dessas categorias para o total.

In [ ]:
def principio_de_pareto(df, col):
    # Calculando a contagem de valores e as porcentagens acumuladas
    value_counts = df[col].value_counts()
    total = value_counts.sum()
    cumsum = value_counts.cumsum()
    cumsum_percentage = cumsum / total * 100
    
    # Definindo o ponto de 80% (incluindo o próximo valor se o anterior não atingir 80%)
    pareto_80 = cumsum_percentage[cumsum_percentage <= 80]
    if pareto_80.empty or cumsum_percentage.iloc[len(pareto_80)] > 80:
        pareto_80 = pd.concat([
            cumsum_percentage[cumsum_percentage <= 80],
            cumsum_percentage[cumsum_percentage > 80].head(1)
        ])
    
    # Plotando o gráfico com duas y-axes (barras e linha de porcentagem acumulada)
    fig, ax1 = plt.subplots(figsize=(18, 6))
    
    # Barras de frequência
    ax1.bar(value_counts.index, value_counts.values, color='blue', alpha=0.7)
    ax1.set_xlabel(f"Valores de {col}")
    ax1.set_ylabel("Frequência", color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # Segundo eixo Y para a porcentagem acumulada
    ax2 = ax1.twinx()
    ax2.plot(cumsum_percentage.index, cumsum_percentage.values, color='red', marker='s', linestyle='-', linewidth=2)
    ax2.set_ylabel("Porcentagem acumulada (%)", color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    ax2.axhline(80, color='green', linestyle='--', label='80%')

    # Adicionar rótulos para os valores percentuais na linha
    for i, v in enumerate(cumsum_percentage.values):
        ax2.text(i, v + 2, f'{v:.1f}%', ha='center', va='bottom', fontsize=10, color='red')

    # Título e layout
    plt.title(f"Diagrama de Pareto para a coluna {col}")
    fig.tight_layout()
    
    # Exibir a legenda
    ax2.legend(loc="best")

    # Mostrar o gráfico
    plt.show()
    
    # Exibindo os valores até pelo menos 80%
    print(f"Para a coluna {col}, os valores que representam 80% dos dados são:")
    print(pareto_80)


A função `principio_de_pareto` foi definida para aplicar o Princípio de Pareto em uma coluna específica do dataframe. Esta função calcula a contagem de valores, a soma cumulativa e a porcentagem cumulativa dos valores em uma coluna. Em seguida, ela gera um gráfico de barras que mostra a contribuição cumulativa das categorias para o total, permitindo identificar quais categorias têm o maior impacto.

Essa função será útil para identificar as categorias mais impactantes nas colunas categóricas do nosso dataset de ocorrências ferroviárias.


Agora iremos aplicar essa função na coluna Classificação Manchete:


In [ ]:
principio_de_pareto(df, "Classificacao Manchete")

Com o gráfico acima, vemos que, 82.2% das ocorrências são causados por ocorrências de segurança pública e usuários, considerando que temos 8 classes e que 2 delas causam mais de 80% das ocorrências, temos um alinhamento com o Princípio de Pareto.

Porém para ter certeza dos nossos resultados podemos realizar uma dupla validação com a coluna Class Manchete:

In [ ]:
principio_de_pareto(df, "Class Manchete")

Como é possível observar, em ambas as colunas, as classificações de ocorrencias "SEGURANÇA PÚBLICA" e "USUÁRIOS" são as mais representativas, com isso, ao serem resolvidos problemas relacionados a ambos, seria possível resolver ao menos 80% dos erros encontrados.

Podemos tambem elencar os top 10 trechos com mais ocorrências, para isso, podemos obsevar o seguinte código:

In [ ]:
trechos = df["Trecho"].value_counts().head(10)

#gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(trechos.index, trechos.values)
plt.title("Top 10 trechos com mais ocorrências")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


Com isso, conseguimos ver que a estação do Braz é responsável pela maior parte das ocorrências. Com isso, podemos descobrir quanto das ocorrencias são no Braz.

In [ ]:
num_ocorrencias = df.shape[0]
braS_ocorrencias = df["Trecho"].value_counts().loc["BAS"]

proporcao_bras = braS_ocorrencias / num_ocorrencias * 100

print(f"A proporção de ocorrências no trecho BAS é de {proporcao_bras:.2f}%")

Isto é, 10% das ocorrencias são no Bras, o que pode se dar pelo auto movimento na via

## Análise Colunas numéricas

Agora iremos analisar as colunas numéricas. Com isso, primeiramente devemos identificá-las

In [ ]:
print("colunas numéricas",list(df.select_dtypes(include=['float64', 'int64']).columns))

Considerando que temos muitas que são "Id" ou "tx", ou seja, provaveis identificações, vamos análisa-los primeiramente visualmente para entender sobre oque falam.

In [ ]:
id_cols = []
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    #se tiver Id no nome da coluna
    if "id" in column.lower() or "tx" in column.lower():
        id_cols.append(column)
        
id_df = df[id_cols]

id_df.head()

Apenas visualmente, conseguimos identificar na tabela, colunas que são referentes a ID's, nelas conseguimos depois  ver por exemplo, em quais localidades há mais ocorrencias, quais trens, tipos de relevancia mais recorrentes, etc. Contudo, por enquanto iremos continuar apenas analisando colunas

Ainda observando as colunas de ID, vemos data id em duas colunas, o que pode ser convertido para uma data em si.

In [ ]:
def converter_uid_data(uid):
    if pd.isnull(uid):
        return pd.NaT
    data_str = str(int(uid))
    return datetime.strptime(data_str, '%Y%m%d')

df['Datanormalizacao'] = df['Id Datanormalizacao'].apply(converter_uid_data)
df['Dataocorrencia'] = df['Id Dataocorrencia'].apply(converter_uid_data)

In [ ]:
df.head()

### Análises dos valores nas colunas numéricas


Observando a tabela, na coluna Id Localidade, conseguimos identificar quais localidades possuiram maior numero de ocorrencias

In [ ]:
df["Id Localidade"].value_counts()


Com isso, é possível observar que a localidade 462 possui um maior numero de ocorrencias, agora, combinando isso com a data das ocorrencias(mês e ano), podemos ver em uma linha do tempo o numero de ocorrencias nessa localidade

In [ ]:
# Filtrando os dados para a localidade 462
df_localidade_462 = df[df["Id Localidade"] == 462]

# Criando uma nova coluna 'AnoMes' para agrupar por mês e ano
df_localidade_462['AnoMes'] = df_localidade_462['Dataocorrencia'].dt.to_period('M')

# Contando o número de ocorrências por mês e ano
ocorrencias_por_mes = df_localidade_462.groupby('AnoMes').size()

# Plotando o gráfico de linha
plt.figure(figsize=(12, 6))
ocorrencias_por_mes.plot(kind='line', marker='o')
plt.title('Número de Ocorrências na Localidade 462 ao Longo do Tempo')
plt.xlabel('Mês e Ano')
plt.ylabel('Número de Ocorrências')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

A visualização no grafico fica um pouco complicada, mas é possível identificar periodos de pico de ocorrências, porém quero afirmar os periodos com ocorrencias acima da média.

In [ ]:
df_localidade_462 = df[df["Id Localidade"] == 462]

# Criando uma nova coluna 'AnoMes' para agrupar por mês e ano
df_localidade_462['AnoMes'] = df_localidade_462['Dataocorrencia'].dt.to_period('M')

# Contando o número de ocorrências por mês e ano
ocorrencias_por_mes = df_localidade_462.groupby('AnoMes').size()

ocorrencias_acima_media = ocorrencias_por_mes[ocorrencias_por_mes > ocorrencias_por_mes.mean()]

ocorrencias_acima_media

Com isso, sabemos que 124 periodos tiveram numero de ocorrencias maior que a média de ocorrencias por mês, sendo o periodo com mais ocorrências no mês 7 de 2009.

Também podemos identificar as horas com mais numero de ocorrência, para isos precisamos converter a hora para o formato 24 horas e então fazer essa verificação com um mapa de calor

In [ ]:
df['Hora Ocorrencia Convertida'] = df['Hora Ocorrencia'].str.replace(",",".") 

df['Hora Ocorrencia Convertida'] = pd.to_numeric(df['Hora Ocorrencia Convertida'], errors='coerce') * 24

df['Hora Ocorrencia Convertida'] = df['Hora Ocorrencia Convertida'].astype(int)

df.head()

hora_ocorrencias = df['Hora Ocorrencia Convertida'].value_counts()

# heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(hora_ocorrencias.to_frame(), annot=True, fmt='d', cmap='viridis')
plt.title('Número de Ocorrências por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Número de Ocorrências')
plt.tight_layout()
plt.show()


Estranhamente, o horario com mais registros é as 23 horas, o que não condiz muito com a movimentação para o horario. Talvez isso se de pelo fato de os registros serem feitos apenas ao final do dia e não ao longo dele.

## Convertendo csv para parquet


Para realizar essa tarefa, o grupo desenvolveu uma função de conversão que será importada abaixo

In [ ]:
from services.parquet import Conversor

conversor = Conversor()

conversor.df_to_parquet(df, "../dados/ocorrencias.parquet")



## Upload de dados para AWS

Para realizar essa tarefa, o grupo desenvolveu uma função de upload que será importada abaixo

In [ ]:
from services.aws_conn import AwsConn

aws_conn = AwsConn()

aws_conn.send_to_s3("ocorrencias.parquet", "big-data-ft-ocorrencias", "../dados/ocorrencias.parquet")